> **DO NOT EDIT IF INSIDE tsds folder**


# Week 6: Spatial data

*Thursday, March 22, 2018*

In this part of today's session you will be working with structuring and plotting spatial data. 
- Exercise 6.1: basic shapes
- Exercise 6.2: working with Geopandas and interpolation
- Exercise 6.3: interactive plotting with folium


**Questions**: Outside of class, use [issue](https://github.com/abjer/tsds/issues) on GitHub for asking questions.

We begin with loading the standard packages:


In [1]:
import os
import requests

import numpy as np
import pandas as pd
import seaborn as sns

import folium
import geopandas as gpd
import fiona
import shapely

%matplotlib inline

## Exercises

### Part 6.1: Working with spatial objects 

This exercise will serve as a brief tutorial on spatial data. We will learn how to make spatial shape and use their basic operations.

> **Ex. 6.1.1 ** Make a polygon which is square that has left,lower corner in 2,2 and right,upper in 5,5. Make a triangle passing through (0,0), (4,0) and (2,4). What do they look like? Store the two polygons respectively as `square` and `triangle`

> *Hint*: the submoduele shapely.geometry has methods such as `Point`, `Polygon` and `Multipolygon`

> **Ex. 6.1.2 ** What is the spatial difference of square subtracted triangle? What is the union?

> **Ex. 6.1.3** Make a GeoSeries of `square`, `triangle`. Plot the geoseries.

### Part 6.2: Working with house sale prices

In this exercise we will work with administrative Danish data and look at house prices.

We have downloaded the shapes of the current Danish municipalities and parishes from [here](https://download.kortforsyningen.dk/content/danmarks-administrative-geografiske-inddeling-110000) for you. They are available [here](https://github.com/abjer/tsds/tree/master/data/dk_admin_shapes/) in the course repo. 

#### Basic operations with Geopandas

The first three exercises in this part will investigate basic concepts and operations with geopandas.

> **Ex. 6.2.1 ** Load the municipalities data. 
- What is the CRS of the GeoDataFrame - what projection does it correspond to? What is the measure of distance? 
- Which three munipalities have the largest area?

> Note: to find the entire area of a municipality which consists of island etc. you can use the `unary_union` method for GeoSeries.

> **Ex. 6.2.2** Select rows in the GeoDataFrame as follows. Make two boolean series as described below and make a combined series which takes the value True if both holds, otherwise False.
> - first: row is True if corresponding the row shape is in the Capital Region or Sealand Region (i.e. `'Region Hovedstaden', 'Region Sjælland'`) 
> - second: row is True if the  the row geometry is ***not*** in Bornholm or nearby (i.e. `'Bornholm', 'Christiansø'`)

> *Hint*: recall that we can check if a series elements are elements in a series using the `isin` method


> **Ex. 6.2.3** Extract the extremum values (min,max) in all dimensions.

> *Hint*: extreme values, i.e. bounds, can be found using `.bounds` on a GeoDataFrame (also works on shapes, GeoSeries) - you further need to compute the global max, min values of these.

#### Interpolation of house prices

In the following three exercises we aim to understand how house price sales are distributed throughout Denmark. We focus on keep our focus on Sealand and nearby. We are interested in seeing the disparity between urban and rural areas. The exercise will illustrate how to make a interpolation of data which is useful for feature engineering and get a good understanding of the data.

To do this we compute local spatial neighborhood measures of house prices in Sealand and around. We first make a grid of squares 500m apart. Then we make a model of house prices and apply the model to infer the prices around Sealand. Finally we visualize the output by selecting only the square intersecting with actual with the administrative data.

> **Ex. 6.2.4** We are now to construct a 500mx500m grid for the bounds of Sealand and around:
- Make a grid of points 500m apart in horizontal and vertical directions that are within the extremum values of Sealand's shape. 


> **Ex. 6.2.5** Compute interpolation of house price for each grid cell as follows:
- Load the pre-structured data with house sales prices for the capital region of Denmark. It is available under the internal course page under files, see [here](https://absalon.instructure.com/courses/24660/files).
- Make a loop over sale_year (2012, 2013):
    - Fit a nearest neighbor regression model to the square meter price (i.e. `price_area` for each year). Set number of neighbors to 25 and radius to 25000        
    - Apply the model to the grid data and assign the estimated price as a column called `pYYYY` where YYYY is the year.
    - Make a new column where you store log10 of house price for the given year and call it`pYYYY_log`

> **Ex 6.3.6:** Visualize the house data
- For each of these points construct a square polygon assuming that the point is the south west corner of the square. 
- Select all sqaures that intersects with Sealand and nearby islands.
- Plot the grid data for 2012

> *Hint:* Once you have created the grid the following function below may be useful for converting into a GeoDataFrame. You need to specify the column names for your x and y coordinates.

> *Hint 2:* We can select the points that intersect by using a spatial join between the house locations and municipalities.m

In [ ]:
from shapely.geometry import box

dk_crs = {'ellps': 'GRS80', 'no_defs': True, 'proj': 'utm', 'units': 'm', 'zone': 32}

def cell_coords_to_polygons(square_df, x='e', y='n', dist=500, crs=dk_crs):
    '''
    Convert coordinates to squares in a GeoDataFrame.
       
    Parameters
    ----------
    x : str
        Name of the horizontal coordinate (~longitude)            
    y : str
        Name of the vertical coordinate (~latitude)                        
    dist : int or float
        Size of polygons
    crs : dict
        Coordinate Reference System


    Returns
    ----------
    squares_gdf: geopandas.GeoDataFrame
        This table contains squares as geometry
        and the original data.
    '''
    
    def _to_square_polygon(row):
        '''
        This auxiliary function convert a square's lower,left 
        coordinates to a polygon. 
        
        Parameters
        ----------
        row : pandas.Series
            This is a DataFrame row.            
        
        Returns
        ----------
        poly: shapely.Polygon        
        
        '''
        
        d = dist/2
        
        poly = box(row[x]-d, row[y]-d, row[x]+d, row[y]+d)
        
        return poly
    
    # convert to polygons
    square_geoms = gpd.GeoSeries(square_df.apply(_to_square_polygon, axis=1), crs=crs)
    
    # make GeoDataFrame
    square_gdf = gpd.GeoDataFrame(data=square_df, geometry=square_geoms)
    
    return square_gdf

# convert example
df_example = pd.DataFrame([(617288, 6049782),(617288, 6050282)], columns=['e','n'])
gdf_example = cell_coords_to_polygons(df_example)

### Part 6.3: Exploring OpenStreetMaps through interactive plotting

In this exercise we will play around with OpenStreetMaps (OSM) data. This exercise is short and exploratory but may serve as the beginning of a deeper analysis of features found in OpenStreetMaps. Before we begin you need to download and unzip the shapefile for Denmark [here](https://download.geofabrik.de/europe/denmark-latest-free.shp.zip). 

> **Ex. 6.3.1.** Load the OSM shapefile for `landuse`. What is the CRS for the OSM data?

> *Pro tip*: You can use python module `os` to lookup all files in a given folder. The command is executed as os.listdir(my_path) where my_path provides the path to the relevant folder.

We are going to focus on the focus in the dataset. We will use the [centroid](https://en.wikipedia.org/wiki/Centroid) of the Polygons which act as points that are easy for plotting.

> **Ex. 6.3.2.** Select the forest features and extract the lat, lon and from centroid.

> **Ex. 6.3.3.** Select the rows that are in Sealand or surrounding isles (you can reuse your answer to 6.2.3). Note make sure that the CRS of the GeoDataFrames match.

> **Ex. 6.3.4.** Make a folium plot of the forest centroids in Sealand or nearby.

> *Hint:* the folium plugin called `FastMarkerCluster` can quickly plot many points. Note another folium plugin is called `MarkerCluster` and can plot points which have names. 

> You can use the below code to get you started with a plot beginning in Copenhagen:

>```
m = folium.Map(location=[55.7, 12.5],
               tiles='Stamen Toner',
               zoom_start=10)
```